# Question Answering IndoBERT IndoSQuAD

Dataset: [Indonesian SQuAD 2.0](https://github.com/Wikidepia/indonesian_datasets/tree/master/question-answering/squad)

In [ ]:
!pip install transformers tensorflow datasets evaluate huggingface_hub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/hugging_face.txt', 'r') as file:
    access_token = file.read()

## Load Dataset

In [ ]:
train = load_dataset('json', data_files='/content/drive/MyDrive/Colab Notebooks/Dataset/Indonesian SQuAD/hf_train-v2.0_small.json', field='data')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train

DatasetDict({
    train: Dataset({
        features: ['answers', 'id', 'question', 'title', 'context'],
        num_rows: 85812
    })
})

In [ ]:
train_dataset = train["train"]
train_dataset[0]

{'answers': {'answer_start': [563], 'text': ['2003']},
 'id': '56be85543aeaaa14008c9066',
 'question': "Kapan Beyonce meninggalkan Destiny's Child dan menjadi penyanyi solo?",
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/bijanse / bee-YON-say) (lah lahir 4 September 1981) adalah penyanyi Amerika, penulis lagu, produser rekaman dan aktris. Lahir dan dibesarkan di Houston, Texas, dia tampil dalam berbagai kompetisi menyanyi dan menari sebagai seorang anak, dan menjadi terkenal pada akhir 1990-an sebagai penyanyi utama dari R&B gadis-grup Destiny\'s Child. Dikelola oleh ayahnya, Mathew Knowles, kelompok itu menjadi salah satu kelompok gadis terlaris di dunia sepanjang masa. Hiatus mereka melihat rilis album debut Beyonce, Berbahaya dalam Cinta (2003), yang didirikan sebagai artis solo di seluruh dunia, memperoleh lima Grammy Awards dan menampilkan Billboard Hot 100 nomor satu "Gila Cinta" dan "Baby Boy."'}

In [ ]:
validation = load_dataset('json', data_files='/content/drive/MyDrive/Colab Notebooks/Dataset/Indonesian SQuAD/hf_dev-v2.0_small.json', field='data')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
validation

DatasetDict({
    train: Dataset({
        features: ['answers', 'id', 'question', 'title', 'context'],
        num_rows: 8170
    })
})

In [ ]:
validation_dataset = validation["train"]
validation_dataset[0]

{'answers': {'answer_start': [270, 270, 270, 270],
  'text': ['Denmark, Islandia dan Norwegia',
   'Denmark, Islandia dan Norwegia',
   'Denmark, Islandia dan Norwegia',
   'Denmark, Islandia dan Norwegia']},
 'id': '56ddde6b9a695914005b962a',
 'question': 'Dari negeri - negeri mana orang Norse berasal?',
 'title': 'Normans',
 'context': 'Normans (Norman: musim hujan; Normands; Latin: Normanni) adalah orang - orang yang pada abad ke - 10 dan ke - 11 memberikan nama mereka ke Normandia, sebuah kawasan di Prancis. Mereka diturunkan dari Norse ("Norman" berasal dari "Norseman") perampok dan bajak laut dari Denmark, Islandia dan Norwegia yang, di bawah pemimpin mereka Rollo, setuju untuk bersumpah setia kepada Raja Charles III dari Francia Barat. Melalui generasi asimilasi dan mencampur dengan penduduk asli Frankis dan Romawi-Gaulis, keturunan mereka secara bertahap akan bergabung dengan Carolingian berbasis Francia. Identitas budaya dan etnis yang berbeda dari orang Norman muncul pada par

## Data Preprocessing

In [ ]:
model_name = 'indolem/indobert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast, tokenizer.padding_side == "right"

(True, True)

In [ ]:
max_seq_length = 384
doc_stride = 128

In [ ]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation="only_second",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length'
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples['answers'][sample_index]

        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
train_features = train_dataset.map(
    prepare_train_features, batched=True, remove_columns=train_dataset.column_names
)

Map:   0%|          | 0/85812 [00:00<?, ? examples/s]

In [ ]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation="only_second",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length'
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = validation_dataset.map(
    prepare_validation_features, batched=True, remove_columns=validation_dataset.column_names
)

Map:   0%|          | 0/8170 [00:00<?, ? examples/s]

## Model Training

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_name, from_pt=True, token=access_token)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 16

train_set = model.prepare_tf_dataset(
    train_features,
    shuffle=True,
    batch_size=batch_size
)

validation_set = model.prepare_tf_dataset(
    validation_features,
    shuffle=False,
    batch_size=batch_size
)

In [ ]:
from transformers import create_optimizer

learning_rate = 2e-5
num_train_epochs = 2
total_train_steps = len(train_set) * num_train_epochs

optimizer, schedule = create_optimizer(
    init_lr=learning_rate, num_train_steps=total_train_steps, num_warmup_steps=0
)

model.compile(optimizer=optimizer, jit_compile=True, metrics=["accuracy"])

model.fit(train_set, epochs=num_train_epochs)

Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
5399/5399 [==============================] - 6523s 1s/step - loss: 1.4757 - end_logits_accuracy: 0.6251 - start_logits_accuracy: 0.5990
Epoch 2/2
5399/5399 [==============================] - 6481s 1s/step - loss: 0.9666 - end_logits_accuracy: 0.7391 - start_logits_accuracy: 0.7168


## Evaluation

In [ ]:
raw_predictions = model.predict(validation_set)

519/519 [==============================] - 210s 394ms/step


In [ ]:
raw_predictions

TFQuestionAnsweringModelOutput(loss=None, start_logits=array([[-7.0937843, -8.821554 , -8.89802  , ..., -9.71634  , -9.698342 ,
        -9.6688385],
       [-1.2828294, -8.334086 , -7.7071376, ..., -9.771024 , -9.802124 ,
        -9.7749815],
       [-5.6912274, -8.429276 , -8.9979105, ..., -9.444508 , -9.615727 ,
        -9.584342 ],
       ...,
       [-6.422336 , -9.133359 , -9.298272 , ..., -9.981219 , -9.963766 ,
        -9.944191 ],
       [-6.7428412, -9.192222 , -9.330319 , ..., -9.724705 , -9.879873 ,
        -9.829022 ],
       [-6.423    , -8.15697  , -8.98952  , ..., -9.746264 , -9.758826 ,
        -9.790595 ]], dtype=float32), end_logits=array([[-6.830151 , -9.178921 , -9.10094  , ..., -9.628869 , -9.654066 ,
        -9.677018 ],
       [-0.3865783, -8.442574 , -8.563218 , ..., -9.33706  , -9.198813 ,
        -9.128974 ],
       [-5.5407147, -8.80259  , -8.7606535, ..., -9.373073 , -9.378365 ,
        -9.427701 ],
       ...,
       [-5.7549605, -8.977046 , -9.287629 , ...

In [ ]:
import numpy as np
import collections
from tqdm.auto import tqdm

def postprocess_qa_predictions(
    examples,
    features,
    all_start_logits,
    all_end_logits,
    n_best_size=20,
    max_answer_length=30
):
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = 0.5
        valid_answers = []
        context = example["context"]

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            # This is what will allow us to map some the positions in our logits to span of texts
            # in the original context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]

            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or
                    # correspond to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or not offset_mapping[start_index]
                        or not offset_mapping[end_index]
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction
            # to avoid failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer
        answer = (best_answer["text"] if best_answer["score"] > min_null_score else "")
        predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(
    validation_dataset,
    validation_features,
    raw_predictions["start_logits"],
    raw_predictions["end_logits"]
)

Post-processing 8170 example predictions split into 8301 features.


  0%|          | 0/8170 [00:00<?, ?it/s]

In [ ]:
final_predictions

OrderedDict([('56ddde6b9a695914005b962a', 'Denmark, Islandia dan Norwegia'),
             ('56ddde6b9a695914005b962b', 'Rollo'),
             ('56ddde6b9a695914005b962c', '10'),
             ('5ad39d53604f3c001a3fe8d1', 'Normans'),
             ('5ad39d53604f3c001a3fe8d2', 'Normandia'),
             ('5ad39d53604f3c001a3fe8d3', 'Rollo'),
             ('5ad39d53604f3c001a3fe8d4', 'paruh pertama abad ke-10'),
             ('56dddf4066d3e219004dad61', 'Kristen'),
             ('5ad3a266604f3c001a3fea2b',
              'prestasi militer mereka yang signifikan dan inovasi'),
             ('56dde0379a695914005b9636', 'Normans/Normanz'),
             ('5ad3ab70604f3c001a3feb89', 'Normans'),
             ('56dde0ba66d3e219004dad75', '911'),
             ('56dde0ba66d3e219004dad76', 'Raja Charles III Francia Barat'),
             ('56dde0ba66d3e219004dad77', 'Seine'),
             ('5ad3ad61604f3c001a3fec0d', 'abad ke - 10'),
             ('5ad3ad61604f3c001a3fec0f', 'Rollo'),
             ('56

In [ ]:
from evaluate import load

metric = load("squad_v2")

In [ ]:
formatted_predictions = [
    {"id": k, "prediction_text": v, "no_answer_probability": 0.0}
    for k, v in final_predictions.items()
]

references = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in validation_dataset
]

metric.compute(predictions=formatted_predictions, references=references)

{'exact': 60.04895960832313,
 'f1': 71.54905859166257,
 'total': 8170,
 'HasAns_exact': 60.04895960832313,
 'HasAns_f1': 71.54905859166257,
 'HasAns_total': 8170,
 'best_exact': 60.04895960832313,
 'best_exact_thresh': 0.0,
 'best_f1': 71.54905859166257,
 'best_f1_thresh': 0.0}

## Push Model and Tokenizer to Huggingface Hub

In [ ]:
model.push_to_hub("indobert-finetuned-indosquad")

tf_model.h5:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
tokenizer.push_to_hub("indobert-finetuned-indosquad")

CommitInfo(commit_url='https://huggingface.co/rickyalan/indobert-finetuned-indosquad/commit/bde0c9ad0c8ec8af9c0a41f377b884743c8b2e89', commit_message='Upload tokenizer', commit_description='', oid='bde0c9ad0c8ec8af9c0a41f377b884743c8b2e89', pr_url=None, pr_revision=None, pr_num=None)